In [23]:
#!g1.1
%pip install transformers
%pip install datasets
# !apt install libomp-dev --yes


import numpy as np
import os
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from datasets import load_dataset, concatenate_datasets
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import DataLoader
from enum import Enum, auto
from tqdm import tqdm

device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device == 'cuda':
    %pip install faiss-gpu -q
else:
    %pip install faiss-cpu -q
    
import faiss
import faiss.contrib.torch_utils

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [24]:
#!g1.1
FLAGS = {
    'batch_size': 16,
    'num_epochs': 1,
    'seed': 1234
}

In [25]:
#!g1.1
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/498 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [26]:
#!g1.1
dataset = load_dataset("code_x_glue_ct_code_to_text", 'python')
train_data, valid_data, test_data = dataset['train'], dataset['validation'], dataset['test']

Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

...

Generating train split:   0%|          | 0/251820 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13914 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/14918 [00:00<?, ? examples/s]

Dataset code_x_glue_ct_code_to_text downloaded and prepared to /tmp/xdg_cache/huggingface/datasets/code_x_glue_ct_code_to_text/python/0.0.0/f8b7e9d51f609a87e7ec7c7431706d4ee0b402e3398560410313d4acc67060a0. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [27]:
#!g1.1
class SeqType(Enum):
  CODE = auto()
  DOC = auto()

    
class TokenizeTransform(object):
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def __call__(self, batch):
        return self.create_one_batch(batch)

    def create_one_batch(self, batch):
        tokens_batch = self.get_formatted_input(batch, SeqType.CODE)
        batch_encoding = self.tokenizer(tokens_batch, padding=True, return_token_type_ids=True, truncation=True)
        code_tokens_ids = batch_encoding.input_ids
        code_token_type_ids = batch_encoding.token_type_ids
        code_attention_mask = batch_encoding.attention_mask
        
        tokens_batch = self.get_formatted_input(batch, SeqType.DOC)
        batch_encoding = self.tokenizer(tokens_batch, padding=True, return_token_type_ids=True, truncation=True)
        doc_tokens_ids = batch_encoding.input_ids
        doc_token_type_ids = batch_encoding.token_type_ids
        doc_attention_mask = batch_encoding.attention_mask
        
        return {'code_tokens_ids': code_tokens_ids,
                'code_token_type_ids': code_token_type_ids,
                'code_attention_mask': code_attention_mask,
                
                'doc_tokens_ids': doc_tokens_ids,
                'doc_token_type_ids': doc_token_type_ids,
                'doc_attention_mask': doc_attention_mask
               }

    def get_formatted_input(self, batch, seq_type):
        if seq_type == SeqType.CODE:
            return self.get_formatted_input_with_f(batch, TokenizeTransform.get_batched_tokens4code)
        elif seq_type == SeqType.DOC:
            return self.get_formatted_input_with_f(batch, TokenizeTransform.get_batched_tokens4docstring)
        else:
            raise Exception("Incorrect sequence type")
            
    def get_formatted_input_with_f(self, batch, get_tokens):
        return [self.tokenizer.cls_token + doc_tokens + self.tokenizer.sep_token + code_tokens + self.tokenizer.sep_token \
                   for doc_tokens, code_tokens in zip(*get_tokens(batch))]

    def get_batched_tokens4code(batch):
        doc_tokens = TokenizeTransform.get_docstring_tokens(batch)
        code_tokens = TokenizeTransform.get_code_tokens(batch)
        return doc_tokens, code_tokens

    def get_batched_tokens4docstring(batch):
        doc_tokens = TokenizeTransform.get_docstring_tokens(batch)
        code_tokens = [''] * len(doc_tokens) 
        return doc_tokens, code_tokens

    def get_docstring_tokens(batch):
        return [' '.join(tokens) for tokens in batch['docstring_tokens']]
    
    def get_code_tokens(batch):
        return [' '.join(tokens) for tokens in batch['code_tokens']]
    
    def rotate_list(lst, shift=1):
        return lst[shift:] + lst[:shift]
    
    
tokenize_transform = TokenizeTransform(tokenizer)

In [28]:
#!g1.1
columns = train_data.column_names

train_dataset = train_data.map(tokenize_transform, batched=True, remove_columns=columns, batch_size=FLAGS['batch_size']).with_format("torch")
valid_dataset = valid_data.map(tokenize_transform, batched=True, remove_columns=columns, batch_size=FLAGS['batch_size']).with_format("torch")
test_dataset = test_data.map(tokenize_transform, batched=True, remove_columns=columns, batch_size=FLAGS['batch_size']).with_format("torch")

Parameter 'function'=<function Dataset.map.<locals>.decorate.<locals>.decorated at 0x7f0be4b9b1f0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/15739 [00:00<?, ?ba/s]

  0%|          | 0/870 [00:00<?, ?ba/s]

/home/jupyter/.local/lib/python3.8/site-packages/dill/_dill.py:1705: PicklingWarning: Cannot locate reference to <enum 'SeqType'>.
  warnings.warn('Cannot locate reference to %r.' % (obj,), PicklingWarning)
/home/jupyter/.local/lib/python3.8/site-packages/dill/_dill.py:1707: PicklingWarning: Cannot pickle <enum 'SeqType'>: __main__.SeqType has recursive self-references that trigger a RecursionError.
  warnings.warn('Cannot pickle %r: %s.%s has recursive self-references that trigger a RecursionError.' % (obj, obj.__module__, obj_name), PicklingWarning)


  0%|          | 0/933 [00:00<?, ?ba/s]

In [29]:
#!g1.1
class FaissKNeighbors:
    def __init__(self, is_cuda):
        self.index = None
        self.is_cuda = is_cuda

    def fit(self, X):
        self.index = faiss.IndexFlatL2(X.shape[1])
        if self.is_cuda:
            res = faiss.StandardGpuResources()
            self.index = faiss.index_cpu_to_gpu(res, 0, self.index)
        self.index.add(X)

    def predict(self, X, k):
        distances, indices = self.index.search(X, k=k)
        return indices

In [53]:
#!g1.1
def eval(model, dataloader, device):
    model.eval()

    batched_code_embs = []
    batched_doc_embs = []

    running_loss = 0
    for iteration, batch in tqdm(enumerate(dataloader)):
        with torch.no_grad():
            with torch.cuda.amp.autocast():
                tokens_ids, token_type_ids, attention_mask = batch['code_tokens_ids'], batch['code_token_type_ids'], batch['code_attention_mask']
                tokens_ids, token_type_ids, attention_mask = tokens_ids.to(device), token_type_ids.to(device), attention_mask.to(device)
                code_embs = model(input_ids=tokens_ids, token_type_ids=token_type_ids, attention_mask=attention_mask).pooler_output

                tokens_ids, token_type_ids, attention_mask = batch['doc_tokens_ids'], batch['doc_token_type_ids'], batch['doc_attention_mask']
                tokens_ids, token_type_ids, attention_mask = tokens_ids.to(device), token_type_ids.to(device), attention_mask.to(device)
                doc_embs = model(input_ids=tokens_ids, token_type_ids=token_type_ids, attention_mask=attention_mask).pooler_output

                neg_code_embs = code_embs.roll(1, 0)

            loss = loss_fn(doc_embs, code_embs, neg_code_embs)
            running_loss += loss.item()

            batched_code_embs.append(code_embs.type(torch.float32))
            batched_doc_embs.append(doc_embs.type(torch.float32))

    concat_code_embs = torch.cat(batched_code_embs, dim=0)
    concat_doc_embs = torch.cat(batched_doc_embs, dim=0)

    loss = running_loss / len(dataloader)

    faiss = FaissKNeighbors(is_cuda=device == torch.device('cuda'))

    k = 1000
    mrrs = []
    for beg_idx in range(0, len(concat_code_embs), k):
        if beg_idx + k > len(concat_code_embs):
            k = len(concat_code_embs) - beg_idx
        doc_embs_subset = concat_doc_embs[beg_idx:beg_idx + k]
        code_embs_subset = concat_code_embs[beg_idx:beg_idx + k]
        faiss.fit(code_embs_subset)  
        preds = faiss.predict(doc_embs_subset, k=k)
        targets = torch.unsqueeze(torch.tensor(range(k)), 1).expand(-1, k).to(device)

        reciprocal_ranks = 1 / (np.argwhere(torch.eq(preds, targets).cpu().numpy())[:,1] + 1)
        mrr_ = np.mean(reciprocal_ranks)
        mrrs.append(mrr_)

    mrrs = torch.tensor(mrrs, device=device)
    return torch.mean(mrrs), loss

In [31]:
#!g1.1
def train(model, dataloader, val_loader, device, epoch, epoch_start):
    model.train()
    running_loss = 0
    for iteration, batch in tqdm(enumerate(dataloader), total=len(dataloader)):
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            tokens_ids, token_type_ids, attention_mask = batch['code_tokens_ids'], batch['code_token_type_ids'], batch['code_attention_mask']
            tokens_ids, token_type_ids, attention_mask = tokens_ids.to(device), token_type_ids.to(device), attention_mask.to(device)
            code_embs = model(input_ids=tokens_ids, token_type_ids=token_type_ids, attention_mask=attention_mask).pooler_output
            
            tokens_ids, token_type_ids, attention_mask = batch['doc_tokens_ids'], batch['doc_token_type_ids'], batch['doc_attention_mask']
            tokens_ids, token_type_ids, attention_mask = tokens_ids.to(device), token_type_ids.to(device), attention_mask.to(device)
            doc_embs = model(input_ids=tokens_ids, token_type_ids=token_type_ids, attention_mask=attention_mask).pooler_output

            neg_code_embs = code_embs.roll(1, 0)
            
        loss = loss_fn(doc_embs, code_embs, neg_code_embs)
        running_loss += loss.item()
        loss.backward()
        # xm.optimizer_step(optimizer)
        optimizer.step()

        if iteration % 500 == 0:
            _loss = running_loss / (iteration + 1)
            elapsed_train_time = time.time() - epoch_start
            print("epoch: {}\titeration: {}\tloss: {}\tthis iteration loss: {}\telapsed time: {}\expected time: {}".\
                  format(epoch, iteration, _loss, loss, elapsed_train_time, len(dataloader) / (iteration + 1) * elapsed_train_time))

    elapsed_train_time = time.time() - epoch_start
    print("\tepoch: {}\ttrain loss: {}\telapsed time: {}".format(epoch, running_loss / len(dataloader), elapsed_train_time))

    torch.save(model, 'fine_tuned_codebert.pt')

    val_mrr, val_loss = eval(model, val_loader, device)
    print("epoch: {}\tvalid loss: {}\tvalid mrr: {}".format(epoch, val_loss, val_mrr))

In [32]:
#!g1.1
  
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=FLAGS['batch_size'],
    drop_last=True)

valid_loader = torch.utils.data.DataLoader(
    valid_dataset,
    batch_size=FLAGS['batch_size'],
    drop_last=True)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=FLAGS['batch_size'],
    drop_last=True)

codebert = AutoModel.from_pretrained("microsoft/codebert-base").to(device).train()
for p in codebert.parameters():
    p.requires_grad = True

learning_rate = 1e-5
optimizer = torch.optim.RMSprop(codebert.parameters(), lr = learning_rate)
loss_fn = torch.nn.TripletMarginLoss(margin=200, p=2)

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

In [51]:
#!g1.1
for epoch in range(FLAGS['num_epochs']):
    epoch_start = time.time()
    train(codebert, train_loader, valid_loader, device, epoch, epoch_start)

  0%|          | 1/15738 [00:00<1:23:15,  3.15it/s]

epoch: 0	iteration: 0	loss: 194.375	this iteration loss: 194.375	elapsed time: 0.321727991104126\expected time: 5063.355123996735


  3%|▎         | 501/15738 [02:18<1:12:55,  3.48it/s]

epoch: 0	iteration: 500	loss: 196.9436127744511	this iteration loss: 195.5	elapsed time: 138.01833772659302\expected time: 4335.594010261718


  6%|▋         | 1001/15738 [04:27<1:07:58,  3.61it/s]

...

epoch: 0	iteration: 15500	loss: nan	this iteration loss: nan	elapsed time: 4123.380038738251\expected time: 4186.423782314856


100%|██████████| 15738/15738 [1:09:45<00:00,  3.76it/s]

	epoch: 0	train loss: nan	elapsed time: 4185.117289066315



869it [01:24, 10.30it/s]


AttributeError: module 'torch' has no attribute 'argwhere'

In [54]:
#!g1.1
val_mrr, val_loss = eval(codebert, valid_loader, device)
print("loss: {}\tmrr: {}".format(val_loss, val_mrr))

869it [01:24, 10.30it/s]
/kernel/fallback/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/kernel/fallback/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


loss: nan	mrr: nan
